In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import seaborn.objects as so
import math
from statsmodels.stats import diagnostic
import statsmodels.stats as st
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.regression.recursive_ls as rls
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pickle

import plotly.express as px

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
import plotly.io as pio
import template_plotly
%load_ext autoreload
%autoreload 2

pd.options.plotting.backend = "plotly"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
df_movie = pd.read_pickle('../data/df_movie.pkl')
df_pairs = pd.read_pickle('../data/df_pairs.pkl')
display(df_movie.sample(2))
display(df_pairs.sample(2))

,Movie_name,Movie_release,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Main_genre,Sec_Genre,Movie_rating,Producer,...,Actor_name,Actor_age_release,Inflation Factor for 2023,2023 valued revenue,Avg_revenue_per_film_at_release,Longevity,Number_of_film_at_release,Avg_rating_per_film_at_release,First_film,Actor_main_genre
296891,First Sunday,2008.0,38608838.0,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",Crime Fiction,Heist,4.6,Matt Alvarez,...,C.J. Sanders,11.0,1.43,5.521064e+07,81670186.0,4.0,2,6.15,False,Musical
356849,The Terminal,2004.0,219417255.0,128.0,"{""/m/064_8sq"": ""French Language"", ""/m/02h40lc""...","{""/m/09c7w0"": ""United States of America""}",Romantic comedy,Comedy-drama,7.4,NaN,...,Kumar Pallana,85.0,1.63,3.576501e+08,145429252.5,3.0,2,7.50,False,Romantic comedy


,Movie_name,Movie_release,Actor_pairs,Movie_revenue,Movie_rating,Actor1,Actor2,Age_difference,Film_count_difference,Average_revenue_difference,First_film,First_film_for_one,Number_of_films_together,Same_genre,Genre
297740,Perfume: The Story of a Murderer,2006.0,"(Paul Berrondo, Sam Douglas)",135039943.0,7.5,Paul Berrondo,Sam Douglas,10.0,0,0.0,True,True,0,True,Thriller
512707,Tropic Thunder,2008.0,"(Amy Stiller, Angela Daun)",188072649.0,7.1,Amy Stiller,Angela Daun,NaN,4,89261274.8,False,True,0,False,None


### RQ1: How does the frequency of actor co-starring impact the chemistry and success of collaborative films?

In [4]:
df_pairs.sort_values(by=['Movie_rating', 'Movie_revenue'], ascending=[False, False], inplace=True)

# Rank the movies based on the sorted order
df_pairs['Rank'] = range(1, len(df_pairs) + 1)

# Calculate the ranking ratio
df_pairs['Ranking_ratio'] = 1 - ((df_pairs['Rank'] - 1) / (len(df_pairs) - 1))

In [6]:
df_pairs.sort_values(['Number_of_films_together', 'RankingRatio'], ascending= [False, False]).head()

,Movie_name,Movie_release,Actor_pairs,Movie_revenue,Movie_rating,Actor1,Actor2,Age_difference,Film_count_difference,Average_revenue_difference,First_film,First_film_for_one,Number_of_films_together,Same_genre,Genre,Rank,RankingRatio
396885,Terror in the Aisles,1984.0,"(Dean Martin, Jerry Lewis)",10004817.0,6.3,Dean Martin,Jerry Lewis,9.0,4,1.515564e+07,False,False,15,True,Comedy,336575,0.390451
294270,Pardners,1956.0,"(Dean Martin, Jerry Lewis)",3600000.0,6.1,Dean Martin,Jerry Lewis,9.0,0,0.000000e+00,False,False,13,True,Comedy,383695,0.305115
34036,Artists and Models,1955.0,"(Dean Martin, Jerry Lewis)",3800000.0,6.5,Dean Martin,Jerry Lewis,9.0,0,0.000000e+00,False,False,11,True,Comedy,291571,0.471955
549956,You're Never Too Young,1955.0,"(Dean Martin, Jerry Lewis)",3400000.0,6.5,Dean Martin,Jerry Lewis,9.0,0,0.000000e+00,False,False,11,True,Comedy,291805,0.471531
171113,Grown Ups,2010.0,"(Adam Sandler, Rob Schneider)",271430189.0,6.0,Adam Sandler,Rob Schneider,3.0,3,7.400058e+06,False,False,11,False,None,388990,0.295525


In [70]:
avg_rating_by_nb_films_together = df_pairs.groupby('Number_of_films_together')['Movie_rating'].mean()
fig = avg_rating_by_nb_films_together.plot(kind='bar')

fig.update_layout(
    title="Average rating by number of films together",
    xaxis_title="Number of films together",
    yaxis_title="Average rating",
    showlegend=False,
    xaxis={'tickmode':'linear'},
    yaxis_range=[4, 8],
)
fig.show()

# Answer :

As we can observe in the plot, the average rating doesn't seem to depend on the frequency of actors costarring.